In [3]:
import sys
import pyzed.sl as sl
import math
import time 
import socket
from datetime import datetime
import matplotlib.pyplot as plt
from pynmeagps.nmeareader import NMEAReader
import utm 
import numpy as np

if __name__ == "__main__":

    # some variables
    camera_pose = sl.Pose()   
    odometry_pose = sl.Pose()    
    py_translation = sl.Translation()
    pose_data = sl.Transform()
    text_translation = ""
    text_rotation = ""  
    lu = ""

    # Create a ZED camera object
    
    init_params = sl.InitParameters(camera_resolution=sl.RESOLUTION.HD720,
                                 coordinate_units=sl.UNIT.METER,
                                 coordinate_system=sl.COORDINATE_SYSTEM.RIGHT_HANDED_Z_UP_X_FWD)
    
    #init_params = sl.InitParameters(camera_resolution=sl.RESOLUTION.HD720,
     #                            coordinate_units=sl.UNIT.METER,
      #                           coordinate_system=sl.COORDINATE_SYSTEM.LEFT_HANDED_Z_UP)
    
    
    #init_params = sl.InitParameters(camera_resolution=sl.RESOLUTION.HD720,
     #                            coordinate_units=sl.UNIT.METER,
      #                           coordinate_system=sl.COORDINATE_SYSTEM.RIGHT_HANDED_Z_UP)
                                 
    # step 1
    # create the camera that will input the position from its odometry
    zed = sl.Camera()
    status = zed.open(init_params)
    if status != sl.ERROR_CODE.SUCCESS:
        print("Camera Open : "+repr(status)+". Exit program.")
        exit()
    
    #set up communication parameters and start publishing
    communication_parameters = sl.CommunicationParameters()
    communication_parameters.set_for_shared_memory()
    zed.start_publishing(communication_parameters)

    # warmup for camera 
    if zed.grab() != sl.ERROR_CODE.SUCCESS:
        print("Camera grab : "+repr(status)+". Exit program.")
        exit()
    else:
        zed.get_position(odometry_pose, sl.REFERENCE_FRAME.WORLD)

    tracking_params = sl.PositionalTrackingParameters()
    fusion_params = sl.PositionalTrackingFusionParameters()
    # These parameters are mandatory to initialize the transformation between GNSS and ZED reference frames.
    tracking_params.enable_imu_fusion = True
    tracking_params.set_gravity_as_origin = True
    err = zed.enable_positional_tracking(tracking_params)
    if (err != sl.ERROR_CODE.SUCCESS):
        print("Camera positional tracking : "+repr(status)+". Exit program.")
        exit()
    camera_info = zed.get_camera_information()

    # step 2
    # init the fusion module that will input both the camera and the GPS
    fusion = sl.Fusion()
    init_fusion_parameters = sl.InitFusionParameters()
    init_fusion_parameters.coordinate_system = sl.COORDINATE_SYSTEM.RIGHT_HANDED_Z_UP_X_FWD
    #init_fusion_parameters.coordinate_system = sl.COORDINATE_SYSTEM.LEFT_HANDED_Z_UP
    #init_fusion_parameters.coordinate_system = sl.COORDINATE_SYSTEM.RIGHT_HANDED_Z_UP
    init_fusion_parameters.coordinate_units = sl.UNIT.METER

    fusion.init(init_fusion_parameters)
    fusion.enable_positionnal_tracking(fusion_params)

    uuid = sl.CameraIdentifier(camera_info.serial_number)
    print("Subscribing to", uuid.serial_number, communication_parameters.comm_type) #Subscribe fusion to camera
    status = fusion.subscribe(uuid, communication_parameters, sl.Transform(0,0,0))
    if status != sl.FUSION_ERROR_CODE.SUCCESS:
        print("Failed to subscribe to", uuid.serial_number, status)
        exit(1)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as stream:
        stream.connect(("10.5.134.174", 5050))
        nmr = NMEAReader(stream)
        (_, parsed_data) = nmr.read()
        init_lat = parsed_data.lat
        init_lon = parsed_data.lon
        init_easting, init_northing, _, _ = utm.from_latlon(init_lat,init_lon)
           
        

        while True:
            #Using mobile phone as a GNSS receiver I have acquired LatLng data here
            (_, parsed_data) = nmr.read()
            lat_ = parsed_data.lat
            lon_ = parsed_data.lon
            easting,northing,_,_ = utm.from_latlon(lat_, lon_)
            
            x = easting- init_easting
            y = northing- init_northing
            
            # get the odometry information
            if zed.grab() == sl.ERROR_CODE.SUCCESS:
                zed.get_position(odometry_pose, sl.REFERENCE_FRAME.WORLD)
        
            elif zed.grab() == sl.ERROR_CODE.END_OF_SVOFILE_REACHED:
                break
    
            #get the GPS information
            gnss_data = sl.GNSSData()
            gnss_data.ts = sl.get_current_timestamp()
        
            # put your GPS corrdinates here : latitude, longitude, altitude
            gnss_data.set_coordinates(lat_, lon_, 0)
        
            # covariance must iterate after each reading, idk why :)
            eph = 0.01
            epv = 0.01
            covariance = [  
                                eph * eph,      0.1,  0.1,
                                0.1,      eph * eph,  0.1,
                                0.1,      0.1,      epv * epv
            ]
            gnss_data.position_covariances = covariance
            fusion.ingest_gnss_data(gnss_data)
        
            # get the fused position
            if fusion.process() == sl.FUSION_ERROR_CODE.SUCCESS:
                fused_tracking_state = fusion.get_position(camera_pose, sl.REFERENCE_FRAME.WORLD)
                if fused_tracking_state == sl.POSITIONAL_TRACKING_STATE.OK:
                        
                    rotation = camera_pose.get_rotation_vector()
                    translation = camera_pose.get_translation(py_translation)
                    text_rotation = str((round(rotation[0], 2), round(rotation[1], 2), round(rotation[2], 2)))
                    #It is me trying to convert the cartesian to lat long, but the output is drifted
                    fus_easting = translation.get()[0] + init_easting
                    fus_northing = translation.get()[1] + init_northing
                    f_lat, f_lon = utm.to_latlon(fus_easting, fus_northing, 44, 'P')
                    print(f_lat, f_lon, fus_easting, fus_northing, lat_, lon_, round(rotation[0], 2), round(rotation[1], 2), round(rotation[2], 2)) 
                    # current_geopose = sl.GeoPose()
                    # current_geopose_satus = fusion.get_geo_pose(current_geopose)

                    # # Retrieve LatLng object
                    # lat_lon = current_geopose.latlng_coordinates()
                    # print(f"Lat: {lat_lon.get_latitude()}, Lon: {lat_lon.get_longitude()}")
                    

    zed.close()


Subscribing to 30507109 COMM_TYPE.INTRA_PROCESS
12.820820332495822 80.03894700132743 395703.4062473746 1417515.167763347 12.8207955 80.0390158333 -0.09 0.42 0.02
12.820820332400626 80.0389470014322 395703.4062587101 1417515.1677527758 12.820818 80.0389558333 -0.09 0.42 0.02
12.820820332352964 80.03894700144198 395703.40625975 1417515.1677475004 12.8207968333 80.0390015 -0.09 0.42 0.02
12.820820332310124 80.03894700145058 395703.40626066364 1417515.167742759 12.8207968333 80.0390015 -0.09 0.42 0.02
12.820820332271383 80.03894700145831 395703.4062614871 1417515.1677384712 12.8207968333 80.0390015 -0.09 0.42 0.02
12.820820332452698 80.03894700089212 395703.4062001125 1417515.1677587535 12.8207968333 80.0390015 -0.09 0.42 0.02
12.820820332385896 80.0389470007274 395703.4061822081 1417515.1677514317 12.8208006667 80.0389841667 -0.09 0.42 0.02
12.820820332409943 80.03894700117827 395703.4062311499 1417515.167753909 12.8208006667 80.0389841667 -0.09 0.42 0.02
12.820820332633089 80.03894700094

KeyboardInterrupt: 